In [ ]:
# Import Statements
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format